In [17]:

from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


DB_PATH = r"C:\python\dataOnline\anaslo_02\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    }

print(spreadSheet_ids.keys())

# 検索キーワードよりホール名取得
SEARCH_WORD = "EXA FIRST"
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    print(f"🔍 '{SEARCH_WORD}' を含むホール名の検索結果:")
    for hall_id, hall_name in results:
        print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
  AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["day"] = df["date"].dt.day
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.weekday

conn.close()

df.head()

dict_keys(['EXA FIRST', 'パラッツォ川越店'])
[('halls',), ('models',), ('prefectures',), ('results',), ('sqlite_sequence',)]
🔍 'EXA FIRST' を含むホール名の検索結果:
 - hall_name: EXA FIRST, hall_id: 108


,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Total_rate,day,month,weekday
0,EXA FIRST,2025-04-12,マイジャグラーV,1001,5743,20,13,-1112,287.2,441.8,174.0,12,4,5
1,EXA FIRST,2025-04-12,マイジャグラーV,1002,1259,1,2,-1065,1259.0,629.5,419.7,12,4,5
2,EXA FIRST,2025-04-12,マイジャグラーV,1003,1371,3,3,-709,457.0,457.0,228.5,12,4,5
3,EXA FIRST,2025-04-12,マイジャグラーV,1004,2374,8,4,-368,296.8,593.5,197.8,12,4,5
4,EXA FIRST,2025-04-12,マイジャグラーV,1005,1813,2,5,-1124,906.5,362.6,259.0,12,4,5


## HOLE_ANALYSIS

In [7]:
# 日ごとにモデル別で表示
SHEET_NAME = "HOLE_ANALYSIS"
df_tmp = df.copy()
# メダル
medals = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
# RB＿RATE
game = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna("")
game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
# WRITE
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, game, row=next_row, include_index=True)

## UNIT_ANALYSIS

In [ ]:
# マシン別メダル
SHEET_NAME = "UNIT_ANALYSIS"
MACHINE_NAME = "マイジャグラーV"
# df_tmp = df[df["model_name"] == MACHINE_NAME].copy()
df_tmp = df.copy()
# メダル数
medals = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# medals["rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
medals.shape
# RB_rate 作成
game = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
# rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna(0)
rb_rate.shape

# 回転数
games = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# games["rank"] = games["Total"].rank(method="min", ascending=False).astype(int)
games.columns = pd.MultiIndex.from_product([["GAMES"], games.columns])
games.head()

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, games, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)

## DAY_DETAIL_N

In [18]:
# 日付指定
DAY = 12
SHEET_NAME = f"DAY_DETAIL_{DAY}"

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()

models = [
    "マイジャグラーV",
    # "ゴーゴージャグラー3",
    # "アイムジャグラーEX-TP",
    # "ファンキージャグラー2",
]

for model in models:

    df_tmp = df[(df["day"] == DAY) & (df["model_name"] == model)].copy()
    df_tmp["date"] = df_tmp["date"].dt.strftime("%Y-%m-%d")

    # メダル数
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["Rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    print(f"{model}_{medals.shape}")

    # RB_RATE
    game = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="game",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    rb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="RB",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    
    rb_rate = (game / rb).round(1)
    medals.drop(labels="Total", level=0, inplace=True)
    rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
    rb_rate = rb_rate.fillna(0)
    print(f"{model}_{rb_rate.shape}")

    # スプレッドシート追記
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5  # 追記行
    set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5  # 追記行

マイジャグラーV_(81, 14)
マイジャグラーV_(82, 14)


## 過去3日のメダル数と翌日のRB確率を表示 → 転記

In [20]:
def medals_and_rb_rate(machine_name, year, month, day, period=3):
    base_date = datetime.date(year, month, day)
    start_date = base_date + datetime.timedelta(days=-1)
    end_date = base_date + datetime.timedelta(days=-period)
    df_tmp = df[
        (df["model_name"] == machine_name)
        & (df["date"].dt.date <= start_date)
        & (df["date"].dt.date >= end_date)
    ]
    df_tmp["date"].unique()

    # メダル
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    medals.tail()

    # RB_RATE
    df_tmp = df[(df["model_name"] == machine_name) & (df["date"].dt.date == base_date)]
    game = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="game",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    bb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="BB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="RB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb_rate = (game / rb).round(1)
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    rb_rate.head()
    
    total_rate = (game / (bb+rb)).round(1)
    total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])

    game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
    bb.columns = pd.MultiIndex.from_product([["BB"], bb.columns])
    rb.columns = pd.MultiIndex.from_product([["RB"], rb.columns])

    # データを表示（長い場合は全表示）
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    medal_rb_rate = pd.concat([medals, game, total_rate, rb_rate, bb, rb], axis=1)
    medal_rb_rate = medal_rb_rate.replace([np.inf, -np.inf], np.nan)
    medal_rb_rate = medal_rb_rate.fillna(0)

    medal_rb_rate = medal_rb_rate[medal_rb_rate["MEDALS"]["rank"] <= 10]

    return medal_rb_rate


# N_DAYS_AGO
MACHINE_NAME = "マイジャグラーV"

medal_rb_rate = medals_and_rb_rate(MACHINE_NAME, 2025, 4, 12)
medal_rb_rate.head(10)

MEDALS                      \
date               2025-04-09 00:00:00 2025-04-10 00:00:00   
model_name unit_no                                           
マイジャグラーV   1011                   -215                -185   
           1015                   -774                -721   
           1019                   -974                -232   
           1026                     -9                -909   
           1044                   -585                -350   
           1050                   -806                -556   
           1079                   -397                -162   
           1083                    441                -638   
           1093                   -653                -265   
           1096                   -650                -456   

                                                                 GAME  \
date               2025-04-11 00:00:00 Total rank 2025-04-12 00:00:00   
model_name unit_no                                                      
マイジャグラーV   1011                  -1159 -1559   10                8429   
           1015                   -409 -1904    9                2671   
           1019                  -1665 -2871    1                8977   
           1026                  -1006 -1924    8                2217   
           1044                  -1747 -2682    4                7571   
           1050                  -1006 -2368    5                4694   
           1079                  -1500 -2059    7                5059   
           1083                  -2568 -2765    3                2707   
           1093                  -1888 -2806    2                4633   
           1096                  -1253 -2359    6                3264   

                            TOTAL_RATE             RB_RATE  \
date               2025-04-12 00:00:00 2025-04-12 00:00:00   
model_name unit_no                                           
マイジャグラーV   1011                  138.2               290.7   
           1015                  205.5               445.2   
           1019                  108.2               272.0   
           1026                  246.3               739.0   
           1044                  128.3               302.8   
           1050                  167.6               312.9   
           1079                  144.5               389.2   
           1083                  112.8               300.8   
           1093                  136.3               289.6   
           1096                  171.8               652.8   

                                    BB                  RB  
date               2025-04-12 00:00:00 2025-04-12 00:00:00  
model_name unit_no                                          
マイジャグラーV   1011                     32                  29  
           1015                      7                   6  
           1019                     50                  33  
           1026                      6                   3  
           1044                     34                  25  
           1050                     13                  15  
           1079                     22                  13  
           1083                     15                   9  
           1093                     18                  16  
           1096                     14                   5

In [21]:
import time
# スプレッドシート追記
target_day = 1
for target_day in range(12, 13):
    # time.sleep(30)
    sheet_name = f"DAY{target_day}"

    models = [
        "マイジャグラーV",
        "ゴーゴージャグラー3",
        "アイムジャグラーEX-TP"
        ]

    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = spreadsheet.worksheet(sheet_name)
    sheet.clear()

    today = datetime.date.today()
    base = datetime.date(today.year, today.month, target_day)
    dates = [base - relativedelta(months=i) for i in range(7)]
    print(dates)

    next_row = 1
    for model in models:
        for d in dates:
            medal_rb_rate = medals_and_rb_rate(model, d.year, d.month, d.day)
            # display(medal_rb_rate)
            set_with_dataframe(sheet, medal_rb_rate, row=next_row, include_index=True)
            existing = get_as_dataframe(sheet, evaluate_formulas=True)
            next_row += medal_rb_rate.shape[0] + 5
    
    print(f"DAY{target_day} 書き込み完了しました")
    


[datetime.date(2025, 4, 12), datetime.date(2025, 3, 12), datetime.date(2025, 2, 12), datetime.date(2025, 1, 12), datetime.date(2024, 12, 12), datetime.date(2024, 11, 12), datetime.date(2024, 10, 12)]
DAY12 書き込み完了しました


## 本日からN日前までのデータを表示

In [14]:
def rb_rate_n_days_ago(start_date, end_date, model_name):
    # RB_RATE
    df_tmp = df[
        (df["model_name"] == model_name)
        & (df["date"].dt.date <= start_date)
        & (df["date"].dt.date >= end_date)
    ].copy()

    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["Rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])

    return medals


today = datetime.date.today()
start_date = today + datetime.timedelta(days=-1)
end_date = today + datetime.timedelta(days=-3)
ndays_ago = rb_rate_n_days_ago(start_date, end_date, model)
ndays_ago.head()

MEDALS                      \
date                  2025-04-10 00:00:00 2025-04-11 00:00:00   
model_name    unit_no                                           
アイムジャグラーEX-TP 1130                   2021               -2762   
              1131                   -121               -1159   
              1132                   -379                  38   
              1133                  -1218                 818   
              1134                   -226                1241   

                                                      
date                  2025-04-12 00:00:00 Total Rank  
model_name    unit_no                                 
アイムジャグラーEX-TP 1130                   -350 -1091    6  
              1131                   -585 -1865    2  
              1132                  -1029 -1370    4  
              1133                   -779 -1179    5  
              1134                   -315   700    9

In [16]:
SHEET_NAME = "nDAYS_AGO"

today = datetime.date.today()
start_date = today + datetime.timedelta(days=-1)
end_date = today + datetime.timedelta(days=-3)

models = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP",
    "ファンキージャグラー2",
]

sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()

next_row = 1
for model in models:
    medals = rb_rate_n_days_ago(start_date, end_date, model)
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5

### 
- 3月5日の過去1日間の出玉ワースト10 → 336以下は5台
- 3月5日の過去2日間の出玉ワースト10 → 336以下は3台
- 3月5日の過去3日間の出玉ワースト10 → 336以下は7台
- 3月5日の過去4日間の出玉ワースト10 → 336以下は6台
### 
- 1月12日の過去1日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去2日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 1月12日の過去5日間の出玉ワースト10 → 336以下は3台
### 
- 2月12日の過去1日間の出玉ワースト10 → 336以下は3台
- 2月12日の過去2日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 2月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去5日間の出玉ワースト10 → 336以下は5台
###
- 3月12日の過去1日間の出玉ワースト10 → 336以下は0台
- 3月12日の過去2日間の出玉ワースト10 → 336以下は1台
- 3月12日の過去3日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 3月12日の過去5日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去6日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去7日間の出玉ワースト10 → 336以下は4台